In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
cd drive/MyDrive/master_thesis/final_code

/content/drive/.shortcut-targets-by-id/16udig9ZMaNcASs5Maj6kv7tg-TL3PnSE/Master Thesis/final_code


In [ ]:
!pip install wandb -qqq

     |████████████████████████████████| 1.7 MB 8.4 MB/s 
     |████████████████████████████████| 170 kB 54.3 MB/s 
     |████████████████████████████████| 133 kB 59.9 MB/s 
     |████████████████████████████████| 97 kB 7.0 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 


In [1]:
import os 
import numpy as np
import matplotlib.pyplot as plt
import shutil
import wandb

from py_files.args import *

In [8]:
wandb.login()

# runs = sorted(os.listdir(path='saved_runs'))   
runs = ["upload_seg_bin_gold_2net_100"]

for run in runs:
    print(run)
    
    ARGS = load_args(run, print_changed=False, silent=True)

    ARGS.train_encoder_seg = True

    # if "reconstruction" in ARGS.name: 
    #     ARGS.training_setup = "reconstruction"
    # else: 
    #     ARGS.training_setup = "segmentation"

    print(ARGS.training_setup)

    mask_losses = pcmra_losses = dice_losses = None
    epochs = None

    if os.path.exists(f'saved_runs/{run}/mask_loss.npy'):
        mask_losses = np.load(f'saved_runs/{run}/mask_loss.npy')
        epochs = mask_losses[-1, 0]

    if os.path.exists(f'saved_runs/{run}/pcmra_loss.npy'):
        pcmra_losses = np.load(f'saved_runs/{run}/pcmra_loss.npy')
        epochs = pcmra_losses[-1, 0]

    if os.path.exists(f'saved_runs/{run}/dice_loss.npy'):
        dice_losses = np.load(f'saved_runs/{run}/dice_loss.npy')
        epochs = dice_losses[-1, 0]

    if os.path.exists(f'saved_runs/{run}/test_results.npy'):
        test_results = np.load(f'saved_runs/{run}/test_results.npy')

    wandb.init(project=f"{ARGS.training_setup}_{ARGS.segmentation}",
               name=f"{ARGS.cnn_setup}_{ARGS.mapping_setup}_{ARGS.first_omega_0}", 
               config=vars(ARGS))
    
    wandb.log({"test_mse_loss": test_results[0], 
               "test_bce_loss": test_results[2], 
               "test_dice_loss": test_results[4]}, step=0)

    if ARGS.training_setup == "reconstruction": 
        for ep, train_mean, _, eval_mean, _ in pcmra_losses: 

            wandb.log({"train_mse_loss": train_mean, 
                       "eval_mse_loss": eval_mean, 
                       "epochs": ep}, step=int(ep))

    if ARGS.training_setup == "segmentation": 
        for (ep, train_bce, _, eval_bce, _), (_, train_dice, _, eval_dice, _) in zip(mask_losses, dice_losses): 
            
            wandb.log({"train_bce_loss": train_bce, 
                       "eval_bce_loss": eval_bce, 
                       "train_dice_loss": train_dice, 
                       "eval_dice_loss": eval_dice, 
                       "epochs": ep}, step=int(ep))
    
    
    wandb.finish()    

upload_seg_bin_gold_2net_100
segmentation


test_mse_loss,0.02499
test_bce_loss,0.03897
test_dice_loss,0.22116
train_bce_loss,0.01988
eval_bce_loss,0.03122
train_dice_loss,0.13719
eval_dice_loss,0.20057
epochs,2650.0
_runtime,1
_timestamp,1630589669
_step,2650


test_mse_loss,▁
test_bce_loss,▁
test_dice_loss,▁
train_bce_loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_bce_loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_dice_loss,██▅▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_dice_loss,██▅▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
